In [1]:
from pynq import Overlay
custom_hw = Overlay("fft.bit")
custom_hw?

In [2]:
dma_config = custom_hw.axi_dma_2
dma_datain = custom_hw.axi_dma_1
dma_dataout = custom_hw.axi_dma_0

In [4]:
dma_config?

In [7]:
dma_datain?

In [8]:
dma_dataout?

In [9]:
import numpy as np
from pynq import allocate

In [10]:
inbuffer = allocate(shape=(8192,), dtype='uint32')
outbuffer = allocate(shape=(8192,), dtype='uint32')
configbuffer = allocate(shape=(1,), dtype='uint32')

In [11]:
fftdata = np.random.randint(50, size=(8192,), dtype=np.uint32)
np.copyto(inbuffer, fftdata)

In [15]:
configbuffer

PynqBuffer([0], dtype=uint32)

In [ ]:
"""check"""

In [16]:
configbuffer[0] = 0b000001110000000000000000 + 13

In [17]:
dma_config.sendchannel.transfer(configbuffer)

In [18]:
dma_config.sendchannel.wait()

In [19]:
%%timeit
dma_datain.sendchannel.transfer(inbuffer)
dma_dataout.recvchannel.transfer(outbuffer)
dma_datain.sendchannel.wait()
dma_dataout.recvchannel.wait()

741 µs ± 600 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
outbuffer

PynqBuffer([      6247,      65507,     655363, ..., 4294901795,
            4293722064, 4293918722], dtype=uint32)

In [20]:
%%timeit
npresult = np.fft.fft(fftdata)

3.76 ms ± 7.88 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
